In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from scipy.stats import boxcox
from statsmodels.api import qqplot
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from pmdarima.utils import diff_inv
from sklearn.metrics import mean_squared_error
import datetime
from datetime import datetime
from tqdm import tqdm
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing

%matplotlib inline
warnings.filterwarnings("ignore")
sns.set(rc = {'figure.figsize': (5, 3)})
plt.style.use("fivethirtyeight")
rand_val = 765

In [2]:
invoice_opportunities = pd.read_excel('Invoice and Opportunities.xlsx')
print(len(invoice_opportunities))
invoice_opportunities.head()

3238


,Type,Reference Nbr.,Customer,Amount,Date,Opportunity ID
0,Invoice,FC0004608,C06507,5000.0,2023-10-10,4504
1,Invoice,FC0004607,C01388,241253.0,2023-10-09,1863
2,Invoice,FC0004605,C19104,83954.0,2023-10-06,5708
3,Invoice,FC0004600,C10528,3000.0,2023-06-23,5617
4,Invoice,FC0004598,C01196,54531.0,2023-10-09,5829


In [3]:
df = invoice_opportunities
new_columns = ['type','reference_number','customer','amount','date','opportunity_id']
df.columns = new_columns
df.head()

,type,reference_number,customer,amount,date,opportunity_id
0,Invoice,FC0004608,C06507,5000.0,2023-10-10,4504
1,Invoice,FC0004607,C01388,241253.0,2023-10-09,1863
2,Invoice,FC0004605,C19104,83954.0,2023-10-06,5708
3,Invoice,FC0004600,C10528,3000.0,2023-06-23,5617
4,Invoice,FC0004598,C01196,54531.0,2023-10-09,5829


In [4]:
df = df.sort_values(by = 'date', ascending = True).reset_index(drop = True)
df.head()

,type,reference_number,customer,amount,date,opportunity_id
0,Invoice,FC0000010,C00621,350000.00,2020-01-01,76
1,Invoice,FC0000007,C00589,3000.00,2020-03-29,1056
2,Invoice,FC0000205,C00036,209.08,2020-04-28,6
3,Invoice,FC0000209,C01533,436988.09,2020-05-07,1666
4,Invoice,FC0000734,C01334,1478.40,2020-08-20,2285


In [5]:
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df['year'] = df['date'].astype('str').str[:4]
df['month'] = df['date'].astype('str').str[:7]
df['year_week'] = df['date'].dt.strftime('%Y-%U')
df.head()

,type,reference_number,customer,amount,date,opportunity_id,year,month,year_week
0,Invoice,FC0000010,C00621,350000.00,2020-01-01,76,2020,2020-01,2020-00
1,Invoice,FC0000007,C00589,3000.00,2020-03-29,1056,2020,2020-03,2020-13
2,Invoice,FC0000205,C00036,209.08,2020-04-28,6,2020,2020-04,2020-17
3,Invoice,FC0000209,C01533,436988.09,2020-05-07,1666,2020,2020-05,2020-18
4,Invoice,FC0000734,C01334,1478.40,2020-08-20,2285,2020,2020-08,2020-33


In [6]:
df = df.groupby(['month'])['amount'].sum().reset_index()
df.head()

,month,amount
0,2020-01,350000.00
1,2020-03,3000.00
2,2020-04,209.08
3,2020-05,436988.09
4,2020-08,1478.40


In [7]:
# We split our dataset to be able to evaluate our models

resultsDict = {}
predictionsDict = {}
forecastsDict = {}

air_pollution = df[['month','amount']]
air_pollution.set_index('month', inplace=True)

split_date = '2023-06'
df_training = air_pollution.loc[air_pollution.index <= split_date]
df_test = air_pollution.loc[air_pollution.index > split_date]
print(f"{len(df_training)} days of training data \n {len(df_test)} days of testing data ")

35 days of training data 
 4 days of testing data 


In [8]:
df_test

,amount
month,
2023-07,3475586.67
2023-08,1368857.00
2023-09,2485528.55
2023-10,495770.00


In [12]:
index = len(df_training)
yhat = list()
for t in tqdm(range(len(df_test.amount))):
    temp_train = air_pollution[:len(df_training)+t]
    model = SimpleExpSmoothing(temp_train.amount)
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
    forecasts = model_fit.forecast()
    yhat = yhat + [predictions]

yhat = pd.concat(yhat)
resultsDict['SES'] = mean_squared_error(df_test.amount, yhat.values)
predictionsDict['SES'] = yhat.values
forecastsDict['SES'] = forecasts

100%|██████████| 4/4 [00:00<00:00, 168.62it/s]


In [13]:
index = len(df_training)
yhat = list()
for t in tqdm(range(len(df_test.amount))):
    temp_train = air_pollution[:len(df_training)+t]
    model = ExponentialSmoothing(temp_train.amount)
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
    forecasts = model_fit.forecast()
    yhat = yhat + [predictions]

yhat = pd.concat(yhat)
resultsDict['HWES'] = mean_squared_error(df_test.amount, yhat.values)
predictionsDict['HWES'] = yhat.values
forecastsDict['HWES'] = forecasts

100%|██████████| 4/4 [00:00<00:00, 159.34it/s]


{'SES': 38    2.145871e+06
 39    2.145871e+06
 40    2.145871e+06
 41    2.145871e+06
 42    2.145871e+06
 43    2.145871e+06
 44    2.145871e+06
 45    2.145871e+06
 46    2.145871e+06
 47    2.145871e+06
 48    2.145871e+06
 49    2.145871e+06
 50    2.145871e+06
 51    2.145871e+06
 dtype: float64,
 'HWES': 38    2.199760e+06
 39    2.199760e+06
 40    2.199760e+06
 41    2.199760e+06
 42    2.199760e+06
 43    2.199760e+06
 44    2.199760e+06
 45    2.199760e+06
 46    2.199760e+06
 47    2.199760e+06
 48    2.199760e+06
 49    2.199760e+06
 50    2.199760e+06
 51    2.199760e+06
 dtype: float64}